In [1]:
import numpy as np
import astropy.io.fits as fits 
import matplotlib.pyplot as plt
from scipy import ndimage
import random
from scipy.optimize import leastsq
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from muse import *

In [2]:
def flux_blr(Hb1_blr_br_data,Hb2_blr_br_data,sigma_eff_Hb1,sigma_eff_Hb2,broad2=False):
    flux1 = np.sum(Hb1_blr_br_data)*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1))#instead of Hb1_blr, we use SII amp1,amp2. 
    flux2 = np.sum(Hb2_blr_br_data)*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2)) 
    if broad2:
        flux = (flux1 + flux2)*(10**(-16))
    else:
        flux = flux1*(10**(-16)) 
    return flux

def flux_nlr(Hb1_nlr_nr_data,Hb2_nlr_nr_data,sigma_eff_Hb1,sigma_eff_Hb2,broad2=False):
    flux1 = np.sum(Hb1_nlr_nr_data)*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb1))
    flux2 = np.sum(Hb2_nlr_nr_data)*(np.sqrt(2.*np.pi)*np.fabs(sigma_eff_Hb2)) 
    if broad2:
        flux_2 = (flux1 + flux2)*(10**(-16))
    else:
        flux_2 = flux1*(10**(-16)) 
    return flux_2

def Hb_blr_lum(z,flux):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    Hb_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux)
    return Hb_lum

def Hb_nlr_lum(z,flux_2):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    Hb_lum2 = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux_2)
    return Hb_lum2
    
def ratio_offset_vel_line_shape(vel_offset,vel_sigma_OIII_br):
    ratio = (vel_sigma_OIII_br/vel_offset)
    return ratio

def line_shape_par(z,wav_min,wav_max,cor_popt):
    c = 3*(10**5)
    wave = np.arange(wav_min,wav_max,0.02) 
    cumsum = np.cumsum(OIII_wo_cont(wave,*cor_popt))
    norm_sum=cumsum/cumsum[-1]
    
    select = (norm_sum>0.1) & (norm_sum<0.9)
    w80_spec = wave[select][-1]-wave[select][0]
    plt.plot(wave[select],norm_sum[select],'-k',label='w80')
    plt.xlim([wav_min+85,wav_max-85])
    plt.ylabel('Flux Density')
    plt.xlabel('Wavelength(Angstrom)')
    plt.legend()
    plt.show()
    
    select =  (norm_sum>0.05) & (norm_sum<0.5)
    v5_spec = wave[select][0]-wave[select][-1]
    plt.plot(wave[select],norm_sum[select],'-b',label='v5')
    plt.xlim([wav_min,wav_max])
    plt.ylabel('Flux Density')
    plt.xlabel('Wavelength(Angstrom)')
    plt.legend()
    plt.show()
    
    [w80,v5] = [((w80_spec)/5006.8)*(c/(1+z)),((v5_spec)/5006.8)*(c/(1+z))]
    return w80,v5

def par_tab(obj,w80,v5,vel_offset,vel_sigma_OIII_br,ratio,flux,Hb_lum,flux_2,Hb_lum2):
    par = [w80,v5,vel_offset,vel_sigma_OIII_br,ratio,flux,Hb_lum,flux_2,Hb_lum2]
    column_names={'w80':0,'v5':1,'vel_offset':2,'vel_sigma_OIII_br':3,'ratio':4,'Hb_blr_flux':5,'log_Hb_lum':6,'Hb_nlr_flux':7,'log_Hb_lum2':8}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[par[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s_par_table.fits'%(obj),overwrite=True)      
    

In [3]:
def algorithm_script(obj,z,broad2):
    (amp_Hb1,amp_Hb2,vel_sigma_Hb1,vel_sigma_Hb2,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,vel_offset) = par(obj)
    (Hb1_blr_br_data,Hb2_blr_br_data) = loadblr('subcube_par_%s.fits'%(obj))
    (Hb1_nlr_nr_data,Hb2_nlr_nr_data) = loadnlr('subcube_par_%s.fits'%(obj))

    sigma_eff_Hb1 = line_width_recons(vel_sigma_Hb1,rest_line=4861.33,inst_res_fwhm=0)
    sigma_eff_Hb2 = line_width_recons(vel_sigma_Hb2,rest_line=4861.33,inst_res_fwhm=0) 
    
    flux = flux_blr(Hb1_blr_br_data,Hb2_blr_br_data,sigma_eff_Hb1,sigma_eff_Hb2,broad2)
    flux_2 = flux_nlr(Hb1_nlr_nr_data,Hb2_nlr_nr_data,sigma_eff_Hb1,sigma_eff_Hb2,broad2)
    
    Hb_lum = Hb_blr_lum(z,flux)
    Hb_lum2 = Hb_nlr_lum(z,flux_2)
    cor_popt = np.array([amp_OIII5007,amp_OIII5007_br,vel_OIII,vel_sigma_OIII,vel_OIII_br,vel_sigma_OIII_br])
    
    (wav_min,wav_max) = wavlim(vel_OIII,vel_OIII_br)
    (w80,v5) = line_shape_par(z,wav_min,wav_max,cor_popt)
    
    ratio = ratio_offset_vel_line_shape(vel_offset,vel_sigma_OIII_br)
    par_tab(obj,w80,v5,vel_offset,vel_sigma_OIII_br,ratio,flux,Hb_lum,flux_2,Hb_lum2)
    return w80,v5

In [4]:
z = {"HE0232-0900":0.043143}
objs = z.keys()

broad2= {'HE0232-0900':False}
for obj in objs:
    algorithm_script(obj,z[obj],broad2[obj])

KeyError: "Key 'amp_OIII5007' does not exist."

In [ ]:
hdu = fits.open('%s_par_table.fits'%(obj))
central_tab = hdu[1].data
central_columns = hdu[1].header
w80 = central_tab.field('w80')[0]
print("W80: ",w80)

vel_offset = central_tab.field('vel_offset')[0] #82.4131

Hb_blr_flux = central_tab.field('Hb_blr_flux')[0] #1.07251e-12
log_Hb_lum = central_tab.field('log_Hb_lum')[0] #42.6697

Hb_nlr_flux = central_tab.field('Hb_nlr_flux')[0] 
log_Hb_lum2 = central_tab.field('log_Hb_lum2')[0] 


Hb_blr_lum = 10**log_Hb_lum
Hb_nlr_lum = 10**log_Hb_lum2
Hb_lum_tot = Hb_blr_lum + Hb_nlr_lum

M_s = 1 #in kg = 1.989*10**30 
n_e = 3942#reference electron density. units are particles/cm^2
M_ionized = (10**7)*(100/n_e)*(Hb_blr_lum/10**41)*M_s 
print("Ionized gas mass in Solar Masses: ",M_ionized)
print("Hb-blr_lum: ",Hb_blr_lum)

print("Hb-nlr_lum: ",Hb_nlr_lum)
print("Total Hb luminosity: ",Hb_lum_tot)

In [ ]:
#Simple conical outflow model
d = 1.234*10**15 #39.928 pc to km
P_kin = (3/2)*M_ionized*(1.989*10**30)*(vel_offset**3)*(1/d)
#M_outflow_s = 3*M_ionized*vel_offset*(1/d)


M_outflow_yr = (3*M_ionized*vel_offset*(1/d)*31536000) #This is in Solar mass/year. In (kg/s): 3*M_ionized*vel_offset*(1/d)
print("Current kinetic power (ergs/s): ",P_kin)


#print("Mass outflow rate in (kg/s): ",M_outflow_s)
print("Mass outflow rate in (Solar Mass/year): ",M_outflow_yr)
momn_outflow_rate = M_outflow_yr*vel_offset
print("Mometntum Outlfow Rate: ",momn_outflow_rate) #THis is just force. 

time = (d/vel_offset)/31536000
print("Time of outflow (in years): ",time)

energy = time*P_kin*31536000
print("Energy of outflow(ergs)",energy)

In [ ]:
#See Moffat for more accurate results. 